In [1]:
from pyspark.sql import SparkSession
import os
from search import Search
from plotting import Plotting


Bad key "text.kerning_factor" on line 4 in
E:\Users\Sam\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def read(spark):
    datasets = {}
    for dataset in os.listdir("./ml-latest-small"):
        data = spark.read.option("header", True).format("csv").load(os.path.join(os.getcwd(),"ml-latest-small",dataset))
        datasets[dataset.split('.')[0]]=data
    return datasets

In [3]:
spark = SparkSession.builder.master("local").appName("Movies").config("conf-key", "conf-value").getOrCreate()
datasets = read(spark)

In [4]:
plotting = Plotting(datasets, spark)
search = Search(datasets, spark)

Search user by id, show the number of movies/genre that he/she has watched

In [5]:
search.search_user_movies('1')

232

In [8]:
search.search_user_genre('1').show()

+---------+--------+
|   genres|count(1)|
+---------+--------+
|    Crime|      45|
|  Romance|      26|
| Thriller|      55|
|Adventure|      85|
|    Drama|      68|
|      War|      22|
|  Fantasy|      47|
|  Mystery|      18|
|  Musical|      22|
|Animation|      29|
|Film-Noir|       1|
|   Horror|      17|
|  Western|       7|
|   Comedy|      83|
| Children|      42|
|   Action|      90|
|   Sci-Fi|      40|
+---------+--------+

